## Esercitazione: Machine Translation

Costruire un modello sequenziale per la traduzione (dall'inglese all'italiano). Il modello prende una sequenza in inglese e torna in output una sequenza in italiano:
1. Costruire le sequenze ed effettuare il padding per entrambe le lingue (NB: le sequenze vanno paddate alla maxlen di entrambe le lingue)
2. Dividi il dataset tra train e test con il 20% di test_size
2. Definire un modello che abbia uno strato di embedding e almeno due strati ricorrenti e in uscita uno strato Dense con il numero di neuroni pari al vocabolario per la traduzione (italiano)
3. NB: Per migliorare le performance sullo strato Dense conviene applicare un layer TimeDistributed in questo modo
        TimeDistributed(Dense())
3. Eseguire l'addestramento per almeno 100 epoche

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("machine_translation.csv")

In [2]:
df.head()

,italian,english
0,tom portò i suoi.,tom brought his.
1,a te non piace il pesce?,don't you like fish?
2,non abbiamo mai riso.,we never laughed.
3,aspetti un momento.,hang on a moment.
4,quando è finito?,when did that end?


In [4]:
english_sentences = df.english.values
italian_sentences = df.italian.values

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(english_sentences,italian_sentences, test_size = 0.2, random_state=1)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

eng_tokenizer = Tokenizer()
ita_tokenizer = Tokenizer()

eng_tokenizer.fit_on_texts(X_train)
ita_tokenizer.fit_on_texts(Y_train)


#sequences
eng_sequences_train = eng_tokenizer.texts_to_sequences(X_train)
ita_sequences_train = ita_tokenizer.texts_to_sequences(Y_train)
eng_sequences_test = eng_tokenizer.texts_to_sequences(X_test)
ita_sequences_test = ita_tokenizer.texts_to_sequences(Y_test)



2022-11-19 08:42:15.260121: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
maxlen_ita = len(max(ita_sequences_train,key=len))
maxlen_eng = len(max(eng_sequences_train,key=len))


#train padding
padded_eng_sentences_train = pad_sequences(eng_sequences_train, padding = 'pre', maxlen = maxlen_eng)
padded_ita_sentences_train = pad_sequences(ita_sequences_train, padding = 'pre', maxlen = maxlen_ita)


#test padding
padded_eng_sentences_test = pad_sequences(eng_sequences_test, padding = 'pre', maxlen = maxlen_eng)
padded_ita_sentences_test = pad_sequences(ita_sequences_test, padding = 'pre', maxlen = maxlen_ita)


In [8]:
italian_vocab_size = len(ita_tokenizer.word_index)+1
english_vocab_size = len(eng_tokenizer.word_index)+1
print("Max Italian sentence length: {}".format(padded_ita_sentences_train.shape[1]))
print("Max English sentence length: {}".format(padded_eng_sentences_train.shape[1]))
print("Italian vocabulary size: {}".format(italian_vocab_size))
print("English vocabulary size: {}".format(english_vocab_size))


Max Italian sentence length: 10
Max English sentence length: 6
Italian vocabulary size: 10005
English vocabulary size: 4932


In [9]:
tmp_x = pad_sequences(padded_eng_sentences_train, maxlen_ita)


In [10]:
tmp_x.shape

(40000, 10)

In [11]:
from keras.backend import clear_session
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, TimeDistributed, Dense, Bidirectional

clear_session()
model = Sequential()
model.add(Embedding(english_vocab_size, 128, input_length=maxlen_ita))
model.add(Bidirectional(LSTM(64, return_sequences=True, activation="tanh")))
model.add(Bidirectional(LSTM(64, return_sequences=True, activation="tanh")))
model.add(TimeDistributed(Dense(italian_vocab_size, activation="softmax")))
model.summary()



2022-11-19 08:42:32.324370: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 128)           631296    
                                                                 
 bidirectional (Bidirectiona  (None, 10, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 10, 128)          98816     
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 10, 10005)        1290645   
 ibuted)                                                         
                                                                 
Total params: 2,119,573
Trainable params: 2,119,573
Non-trainable params: 0
______________________________________________

In [12]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [13]:
model.fit(tmp_x, padded_ita_sentences_train, batch_size=512, epochs=180,validation_split=.20)

Epoch 1/180
63/63 [==============================] - 61s 898ms/step - loss: 3.7523 - accuracy: 0.6417 - val_loss: 2.4695 - val_accuracy: 0.6536
Epoch 2/180
63/63 [==============================] - 57s 901ms/step - loss: 2.3789 - accuracy: 0.6549 - val_loss: 2.3395 - val_accuracy: 0.6615
Epoch 3/180
63/63 [==============================] - 57s 908ms/step - loss: 2.2864 - accuracy: 0.6636 - val_loss: 2.2903 - val_accuracy: 0.6664
Epoch 4/180
63/63 [==============================] - 57s 900ms/step - loss: 2.2238 - accuracy: 0.6678 - val_loss: 2.2517 - val_accuracy: 0.6694
Epoch 5/180
63/63 [==============================] - 57s 911ms/step - loss: 2.1753 - accuracy: 0.6699 - val_loss: 2.2133 - val_accuracy: 0.6728
Epoch 6/180
63/63 [==============================] - 56s 897ms/step - loss: 2.1320 - accuracy: 0.6726 - val_loss: 2.1736 - val_accuracy: 0.6754
Epoch 7/180
63/63 [==============================] - 57s 901ms/step - loss: 2.0929 - accuracy: 0.6759 - val_loss: 2.1349 - val_accuracy:

Epoch 58/180
63/63 [==============================] - 56s 885ms/step - loss: 0.7311 - accuracy: 0.8317 - val_loss: 1.1104 - val_accuracy: 0.7922
Epoch 59/180
63/63 [==============================] - 57s 908ms/step - loss: 0.7184 - accuracy: 0.8337 - val_loss: 1.1060 - val_accuracy: 0.7892
Epoch 60/180
63/63 [==============================] - 57s 904ms/step - loss: 0.7066 - accuracy: 0.8351 - val_loss: 1.1014 - val_accuracy: 0.7935
Epoch 61/180
63/63 [==============================] - 56s 896ms/step - loss: 0.6950 - accuracy: 0.8370 - val_loss: 1.0938 - val_accuracy: 0.7927
Epoch 62/180
63/63 [==============================] - 57s 906ms/step - loss: 0.6834 - accuracy: 0.8386 - val_loss: 1.0832 - val_accuracy: 0.7937
Epoch 63/180
63/63 [==============================] - 57s 904ms/step - loss: 0.6719 - accuracy: 0.8406 - val_loss: 1.0804 - val_accuracy: 0.7952
Epoch 64/180
63/63 [==============================] - 57s 906ms/step - loss: 0.6602 - accuracy: 0.8422 - val_loss: 1.0877 - val_ac

63/63 [==============================] - 61s 964ms/step - loss: 0.2259 - accuracy: 0.9141 - val_loss: 1.0349 - val_accuracy: 0.8063
Epoch 171/180
63/63 [==============================] - 66s 1s/step - loss: 0.2244 - accuracy: 0.9144 - val_loss: 1.0357 - val_accuracy: 0.8049
Epoch 172/180
63/63 [==============================] - 65s 1s/step - loss: 0.2238 - accuracy: 0.9142 - val_loss: 1.0348 - val_accuracy: 0.8063
Epoch 173/180
63/63 [==============================] - 67s 1s/step - loss: 0.2227 - accuracy: 0.9146 - val_loss: 1.0364 - val_accuracy: 0.8054
Epoch 174/180
63/63 [==============================] - 72s 1s/step - loss: 0.2212 - accuracy: 0.9147 - val_loss: 1.0422 - val_accuracy: 0.8091
Epoch 175/180
63/63 [==============================] - 71s 1s/step - loss: 0.2201 - accuracy: 0.9150 - val_loss: 1.0397 - val_accuracy: 0.8077
Epoch 176/180
63/63 [==============================] - 72s 1s/step - loss: 0.2195 - accuracy: 0.9148 - val_loss: 1.0459 - val_accuracy: 0.8085
Epoch 177/

In [14]:
test_x = pad_sequences(padded_eng_sentences_test, maxlen_ita)
model.evaluate(test_x,padded_ita_sentences_test)

313/313 [==============================] - 12s 38ms/step - loss: 0.9975 - accuracy: 0.8099


[0.9975460171699524, 0.8099499940872192]

In [15]:
emb_preds = model.predict(test_x)
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

for i in range(0,20):
    print(X_test[i])
    print(logits_to_text(emb_preds[i],ita_tokenizer))
    print("----------------------------------------------------------\n")

313/313 [==============================] - 10s 27ms/step
tom is awake.
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> tom è sveglio
----------------------------------------------------------

i just did it.
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> io l'ho appena fatto
----------------------------------------------------------

tom is brilliant.
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> tom è brillante
----------------------------------------------------------

allow me to help.
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> deve aiutare
----------------------------------------------------------

we heard tom.
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> sentito tom
----------------------------------------------------------

don't waste my time.
<PAD> <PAD> <PAD> <PAD> <PAD> si del i mio tempo
----------------------------------------------------------

you were sick.
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> eri malata
----------------------------------------------------------

tom play